In [ ]:
# useful additional packages
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import time
import dimod
from qiskit.circuit.library import TwoLocal
from qiskit_optimization.applications import Maxcut, Tsp
from qiskit_algorithms import SamplingVQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import SPSA
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import Sampler
from qiskit.primitives import StatevectorSampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from itertools import permutations
from qiskit_aer import Aer, AerSimulator
from qiskit import transpile
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_optimization.converters import QuadraticProgramToQubo

In [ ]:
def draw_graph(G, colors, pos):
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(G, node_color=colors, node_size=600, alpha=0.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)
# Generating a graph of 3 nodes
n = 4
num_qubits = n**2
tsp = Tsp.create_random_instance(n, seed=123)
adj_matrix = nx.to_numpy_array(tsp.graph)
print("Distance\n", adj_matrix)

colors = ["r" for node in tsp.graph.nodes]
pos = [tsp.graph.nodes[node]["pos"] for node in tsp.graph.nodes]

# Timing the brute force TSP computation
start_time = time.time()
#best_distance, best_order = brute_force_tsp(adj_matrix, n)
end_time = time.time()
elapsed_time = end_time - start_time
#print(f"Elapsed time: {elapsed_time:.4f} seconds")
draw_graph(tsp.graph, colors, pos)

In [ ]:
from itertools import permutations
def brute_force_tsp(w, N):
    a = list(permutations(range(1, N)))
    last_best_distance = 1e10
    for i in a:
        distance = 0
        pre_j = 0
        for j in i:
            distance = distance + w[j, pre_j]
            pre_j = j
        distance = distance + w[pre_j, 0]
        order = (0,) + i
        if distance < last_best_distance:
            best_order = order
            last_best_distance = distance
            print("order = " + str(order) + " Distance = " + str(distance))
    return last_best_distance, best_order
best_distance, best_order = brute_force_tsp(adj_matrix, n)
print(
    "Best order from brute force = "
    + str(best_order)
    + " with total distance = "
    + str(best_distance)
)
def draw_tsp_solution(G, order, colors, pos):
    G2 = nx.DiGraph()
    G2.add_nodes_from(G)
    n = len(order)
    for i in range(n):
        j = (i + 1) % n
        G2.add_edge(order[i], order[j], weight=G[order[i]][order[j]]["weight"])
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(
        G2, node_color=colors, edge_color="b", node_size=600, alpha=0.8, ax=default_axes, pos=pos
    )
    edge_labels = nx.get_edge_attributes(G2, "weight")
    nx.draw_networkx_edge_labels(G2, pos, font_color="b", edge_labels=edge_labels)
# Timing the brute force TSP computation
start_time = time.time()
end_time = time.time()
elapsed_time = end_time - start_time
#print(f"Elapsed time: {elapsed_time:.4f} seconds")
draw_tsp_solution(tsp.graph, best_order, colors, pos)

In [ ]:
qp = tsp.to_quadratic_program()
#print(qp.prettyprint())

In [ ]:
from qiskit_optimization.converters import QuadraticProgramToQubo

qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(qp)
qubitOp, offset = qubo.to_ising()
#print("Offset:", offset)
#print("Ising Hamiltonian:")
#print(str(qubitOp))

In [ ]:
#result = exact.solve(qubo)
#print(result.prettyprint())

In [ ]:
# Making the Hamiltonian in its full form and getting the lowest eigenvalue and eigenvector
ee = NumPyMinimumEigensolver()
result = ee.compute_minimum_eigenvalue(qubitOp)

#print("energy:", result.eigenvalue.real)
#print("tsp objective:", result.eigenvalue.real + offset)
x = tsp.sample_most_likely(result.eigenstate)
print("feasible:", qubo.is_feasible(x))
z = tsp.interpret(x)
print("solution:", z)
print("solution objective:", tsp.tsp_value(z, adj_matrix))
#draw_tsp_solution(tsp.graph, z, colors, pos)

In [ ]:
algorithm_globals.random_seed = 123
seed = 10598

optimizer = SPSA(maxiter=300)
ry = TwoLocal(qubitOp.num_qubits, "ry", "cz", reps=5, entanglement="linear")
#sampler = Aer.get_backend('qasm_simulator')
vqe = SamplingVQE(sampler=Sampler(), ansatz=ry, optimizer=optimizer)

result = vqe.compute_minimum_eigenvalue(qubitOp)

#print("energy:", result.eigenvalue.real)
print("time:", result.optimizer_time)
x = tsp.sample_most_likely(result.eigenstate)
#print("feasible:", qubo.is_feasible(x))
z = tsp.interpret(x)
print("solution:", z)
print("solution objective:", tsp.tsp_value(z, adj_matrix))
draw_tsp_solution(tsp.graph, z, colors, pos)

In [ ]:
algorithm_globals.random_seed = 123
seed = 10598

In [ ]:
# create minimum eigen optimizer based on SamplingVQE
vqe_optimizer = MinimumEigenOptimizer(vqe)

# solve quadratic program
result = vqe_optimizer.solve(qp)
#print(result.prettyprint())

z = tsp.interpret(x)
print("solution:", z)
print("solution objective:", tsp.tsp_value(z, adj_matrix))
draw_tsp_solution(tsp.graph, z, colors, pos)

# Timing the brute force TSP computation
start_time = time.time()
end_time = time.time()
elapsed_time = end_time - start_time
#print(f"Elapsed time: {elapsed_time:.4f} seconds")

# Merge one

In [ ]:
def solve_tsp(n):
    start_time = time.time()
    
    # Generating a graph with n nodes
    tsp = Tsp.create_random_instance(n, seed=123)
    adj_matrix = nx.to_numpy_array(tsp.graph)
    print("distance\n", adj_matrix)
    # Visualizing the graph
    colors = ["r" for node in tsp.graph.nodes]
    pos = [tsp.graph.nodes[node]["pos"] for node in tsp.graph.nodes]
    draw_graph(tsp.graph, colors, pos)
    
    # Converting TSP to Quadratic Program and QUBO
    qp = tsp.to_quadratic_program()
    #print(qp.prettyprint())
    qp2qubo = QuadraticProgramToQubo()
    qubo = qp2qubo.convert(qp)
    qubitOp, offset = qubo.to_ising()
    #print("Offset:", offset)
    #print("Ising Hamiltonian:")
    #print(str(qubitOp))
    
    # Solving the QUBO using NumPyMinimumEigensolver (Classical Algorithm)
    ee = NumPyMinimumEigensolver()
    result = ee.compute_minimum_eigenvalue(qubitOp)
    #print("energy:", result.eigenvalue.real)
    #print("tsp objective:", result.eigenvalue.real + offset)
    x = tsp.sample_most_likely(result.eigenstate)
    #print("feasible:", qubo.is_feasible(x))
    z = tsp.interpret(x)
    #print("solution:", z)
    #print("solution objective:", tsp.tsp_value(z, adj_matrix))
    draw_tsp_solution(tsp.graph, z, colors, pos)
    
    
    
    
    # Using VQE (Variational Quantum Eigensolver) to Solve the Problem(Quantum Algrithm)
    algorithm_globals.random_seed = 123
    seed = 10598
    optimizer = SPSA(maxiter=300)
    ry = TwoLocal(qubitOp.num_qubits, "ry", "cz", reps=5, entanglement="linear")
    vqe = SamplingVQE(sampler=Sampler(), ansatz=ry, optimizer=optimizer)
    result = vqe.compute_minimum_eigenvalue(qubitOp)
    #print("energy:", result.eigenvalue.real)
    print("Time spent by the optimization algorithm:", result.optimizer_time)
    x = tsp.sample_most_likely(result.eigenstate)
    print("feasible:", qubo.is_feasible(x))
    z = tsp.interpret(x)
    #print("solution:", z)
    #print("solution objective:", tsp.tsp_value(z, adj_matrix))
    draw_tsp_solution(tsp.graph, z, colors, pos)
    end_time = time.time()
    print(f"Total time for solving TSP with n={n}: {end_time - start_time} seconds")

# Example usage with n = 4
solve_tsp(3)